In [71]:
import os
import textwrap

import csv
import xlrd
from datetime import datetime
import mechanize as mech, configparser

import glob 

files = glob.glob("data/*.xls")


xls_files = [ (int(f[12:16]), int(f[16:18]), f)  for f in files]
xls_files[0]

(1993, 6, 'data\\TOP500_199306.xls')

In [72]:
print("Reconciling headers...")

all_headers = ['Year', 'Month', 'Day']
last_headers = []
headers_to_rename = {
    'Rmax':'RMax', 
    'Rpeak':'RPeak', 
    'Effeciency (%)':'Efficiency (%)',
    'Proc. Frequency': 'Processor Speed (MHz)',
    'Cores': 'Total Cores',
    "Power Effeciency [GFlops/Watts]" : 'Power Efficiency [GFlops/Watts]',
    "Accelerator/Co-Processor Cores" : "Accelerator Cores",
    "Cores per Socket" : "Processor Cores", 
}

for (year, month, fn) in xls_files:
    w = xlrd.open_workbook(fn, logfile=open(os.devnull, 'w'))
    s = w.sheets()[0]
    for rr in range(s.nrows):
        r = s.row_values(rr)
        if any(r): #skip blank rows
            renamed_headers = {}
            headers = [(renamed_headers.setdefault(h, headers_to_rename[h]) if h in headers_to_rename else h)
                       for h in r]
            new_headers = [h for h in headers if h not in all_headers]
            drop_headers = [h for h in last_headers if h not in headers]

            if new_headers or drop_headers or renamed_headers:
                print("{}/{}:".format(year,month))
                if renamed_headers:
                    print(textwrap.fill("renamed headers: " + ', '.join('%s to %s'%kv for kv in renamed_headers.items()), initial_indent='  ', subsequent_indent='    '))
                if new_headers:
                    print(textwrap.fill("new headers: " + ', '.join(new_headers), initial_indent='  ', subsequent_indent='    '))
                if drop_headers:
                    print(textwrap.fill("drop headers: " + ', '.join(drop_headers), initial_indent='  ', subsequent_indent='    '))

            all_headers.extend(new_headers)
            last_headers = headers
            break
    del w, s


Reconciling headers...
1993/6:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
  new headers: Rank, Site, Manufacturer, Computer, Country,
    Processors, RMax, RPeak, Nmax, Nhalf, Processor Family, Processor,
    Processor Speed (MHz), System Family, Operating System,
    Architecture, Segment, Application Area, Interconnect Family,
    Interconnect, Region, Continent
1993/11:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1994/6:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1994/11:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1995/6:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1995/11:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1996/6:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1996/11:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1997/6:
  renamed headers: Proc. Frequency to Processor Speed (MHz)
1997/11:
  renamed headers: Proc. Frequency to Processor Speed (MHz)

In [73]:
# Second pass, build a complete CSV table
print("Building complete TOP500_history.csv...")


with open("TOP500_history.csv", "w", encoding="utf-8") as history :
    out = csv.DictWriter(history, all_headers)
    out.writeheader()

    for (year,month,fn) in xls_files:
        w = xlrd.open_workbook(fn, logfile=open(os.devnull, 'w'))    
        s = w.sheets()[0]
        headers = None
        for rr in range(s.nrows):
            r = s.row_values(rr)
            if any(r): #skip blank lines
                if headers is None:
                    headers = [headers_to_rename.get(h,h) for h in r]
                else:
                    rd = dict(zip(headers, r))
                    rd.update({'Year':year, 'Month':month, 'Day':1})
                    out.writerow(rd)
        del w, s

print("Done.")

Building complete TOP500_history.csv...
Done.


In [74]:
import pandas as pd


def remap(procfam):
    if procfam in ('Intel EM64T','Intel Nehalem','Intel Westmere','Intel SandyBridge','Intel IvyBridge','Intel Haswell','Intel Core','Intel Broadwell','Intel Skylake','Intel Cascade Lake','Intel Cascade lake','AMD x86_64','AMD Zen (Naples)','AMD Zen-2 (Rome)'):
        i,v='x86-64', procfam.split()[0]
    elif procfam in ('Intel MIC','Intel Xeon Phi'):
        i,v='Xeon Phi','Intel'
    elif procfam in ('POWER','Power','PowerPC'):
        i=v='POWER'
    elif procfam == 'Intel IA-64':
        i,v='Itanium', 'Intel'
    elif procfam in ('Intel IA-32','AMD'):
        i,v='x86-32', procfam.split()[0]
    else:
        i,v=procfam, procfam
    return pd.Series((i,v))


df = pd.read_csv("TOP500_history.csv")


procfam = df['Processor Family'].where(df['Processor Family'].notna(), df['Processor Technology'])
df[['ISA','Vendor']] = procfam.apply(remap)


# get country codes
for f, t in (('Saudia Arabia', 'Saudi Arabia'),     # typo in TOP500 sources
             ('Korea, South', 'South Korea'),       # Match country-en.csv
             ('Czech Republic', 'Czechia'),         # Match country-en.csv
             ('Slovak Republic', 'Slovakia'),       # Match country-en.csv
             ('Hong Kong', 'Hong Kong SAR China')): # Match country-en.csv
    df['Country'].replace(f, t, inplace=True)
dfc_en = pd.read_csv('data/country-en.csv')
dfc_en.columns = ('CountryISO', 'Country')
df = df.merge(dfc_en, on='Country', how='left')
assert (df['CountryISO'].isnull()==False).all()


df.to_csv("TOP500_all.csv", index=False)

C:\Users\a791670\Anaconda3\envs\hpcmarket\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (30,31,35,36,37,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
